In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
from tensorflow.keras.models import Model, load_model, model_from_json
from PIL import Image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from random import choice,shuffle
from PIL import Image
from scipy import stats as st
from collections import deque,Counter
import tensorflow as tf

In [2]:
#model = load_model(r"C:\Users\Shivam\Desktop\dl_project\cricket\detector_3.h5")
json_file = open(r"C:\Users\Shivam\Desktop\dl_project\cricket\detector_10.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
# load weights into new model
model.load_weights(r"C:\Users\Shivam\Desktop\dl_project\cricket\detector_10.h5")
print("Loaded model from disk")

Loaded model from disk


In [3]:
def show_winner(user_score, computer_score):   
    print("\nTotal User Score : ",user_score)
    print("Total Computer Score : ",computer_score)
    if user_score > computer_score:
        print("\nUser won")
        img = cv2.imread("won1.jpg")
        img = cv2.resize(img,(640,480),Image.ANTIALIAS)
        
    elif user_score < computer_score:
        img = cv2.imread("loss1.jpg")
        img = cv2.resize(img,(640,480),Image.ANTIALIAS)
        print("\nComputer won")
    else:
        img = cv2.imread("tie1.jpg")
        img = cv2.resize(img,(640,480),Image.ANTIALIAS)
        print("\nTie")
        
    cv2.putText(img, "Press 'ENTER' to play again, else press 'Q' for exit",
                    (40, 400), cv2.FONT_HERSHEY_COMPLEX, 0.65, (34,139,34), 2, cv2.LINE_AA)
    cv2.imshow("Cricket Game", img)
        
    # If enter is pressed.
    k = cv2.waitKey(0)
    # If the user presses 'ENTER' key then return TRUE, otherwise FALSE
    if k == 13:
        return True
    else:
        return False


In [7]:
frame.shape

(480, 640, 3)

In [23]:
cap = cv2.VideoCapture(0)
repeat=0
box_size = 234
width = int(cap.get(3))
# Specify the number of attempts you want.
attempts = 6
# Initially the moves will be `nothing`
computer_move_name= "nothing"
final_user_move = "nothing"
label_names = ['0','1','2','3','4','5','6']
# All scores are 0 at the start.
computer_score, user_score = 0, 0
# The default color of bounding box is Blue
rect_color = (255, 0, 0)
# This variable remembers if the hand is inside the box or not.
hand_inside = False
# At each iteration we will decrease the total_attempts value by 1
total_attempts = attempts
# We will only consider predictions having confidence above this threshold.
confidence_threshold = 0.90
# Instead of working on a single prediction, we will take the mode of 7 predictions by using a deque object
# This way even if we face a false positive, we would easily ignore it, by taking the mode i.e.selecting the  
# one with maximum frequency
smooth_factor = 15
# Our initial deque list will have 'nothing' repeated 7 times.
de = deque(['nothing'] * 15, maxlen=smooth_factor)
# This will store the runs scored in every 6 attempts and will help in alerting user to not repeat same number of scores
# using counter
over = []
count_balls = Counter(over)
n=0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
  # extract the region of image within the user rectangle
    roi = frame[5: box_size-5 , width-box_size + 5: width -5]
    roi = cv2.resize(roi,(310,310),Image.ANTIALIAS)
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    roi = np.array([roi]).astype('float32') / 255.0
    roi = roi.reshape(1,310,310,1)    
    # Predict the move made
    pred = model.predict(roi)
   # Get the index of the predicted class
    move_code = np.argmax(pred[0])
   # Get the class name of the predicted class
    user_move = label_names[move_code]
    # Get the confidence of the predicted class
    prob = np.max(pred[0])
    cv2.putText(frame, "Prediction: {} {:.2f}%".format(label_names[np.argmax(pred[0])], prob*100 ),
                (10, 40), cv2.FONT_HERSHEY_TRIPLEX, 0.90, (0, 0, 255), 2, cv2.LINE_AA)
    # Make sure the probability is above our defined threshold
    if prob >= confidence_threshold:
       # Now add the move to deque list from left
        de.appendleft(user_move)
        # Get the mode i.e. which class has occured more frequently in the last 5 moves.
        try:
            final_user_move = st.mode(de)[0][0]
        except StatisticsError:
            print('Stats error')
            continue
            
        # The model will not detect anything if there is no user move i.e. only background is visible
        # therfore won't go further in 'if' statement
        # so the else part will work where we make 'hand_inside' to be False
        # thus 'if' statement will work now if we show some finger to camera
        # in this 'if' statement we then enable 'hand_inside' to be true 
        # So now the user has to take his hands out of the box for every new prediction.
        # Basically the hand_inside variable is helping us to not repeatedly predict during the loop
        if final_user_move != '0':

            if final_user_move != 'nothing' and hand_inside == False:
                over.append(final_user_move)
                count_balls = Counter(over)
                # This helps in checking if the user has repeatedly shown same fingers in an over (a particular no. of times)
                # For our game we used 2 as maximum repetition per over, beyond which alert will be prompted to show 
                # different finger
                if count_balls[final_user_move]>2:
                    print("\nPlease do not repeat number")
                    
                    cv2.putText(frame,"Please do not repeat number",
                            (100, 450), cv2.FONT_HERSHEY_TRIPLEX, 0.65, (255,255,255), 1, cv2.LINE_AA)                    
                    # Set hand inside to True
                    hand_inside = True
                    
                else:
                    # This tracks the balls in the over
                    n+=1
                    print("\nBall {}".format(attempts-total_attempts+1))
                    # Set hand inside to True
                    hand_inside = True
                   # Get Computer's move 
                    computer_move_name = choice(['1','2','3','4','5','6'])

                    # Display the computer's move
                    #display_computer_move(computer_move_name, frame)
                   
                    # Subtract one attempt
                    total_attempts -= 1
                    
                    # Print user and computer move
                    print("User move : ",final_user_move)
                    print("Computer move : ",computer_move_name)
                    
                    # This is a condition that will run at the end of every over, so that other conditions remain operative
                    if n==6:
                        over=[]
                        n=0
                                                
                    # Rule of game, if user move and computer move gets same, the game is terminated without results.
                    if final_user_move == computer_move_name:
                        print("\nSame Score, Computer Won\n")
                        img = cv2.imread("same_score.png")
                        img = cv2.resize(img,(640,480),Image.ANTIALIAS)

                        cv2.putText(img, "Press 'ENTER' to play again, else press 'Q' for exit",
                        (40, 450), cv2.FONT_HERSHEY_COMPLEX, 0.65, (34,139,34), 2, cv2.LINE_AA)
                        cv2.putText(img,"Computer Won, You could not last long",
                        (100, 430), cv2.FONT_HERSHEY_COMPLEX, 0.65, (34,139,34), 2, cv2.LINE_AA)
                        cv2.imshow("Cricket Game", img)

                        k = cv2.waitKey(0)
                        if k == 13 or k==10:
                            user_score, computer_score, total_attempts = 0, 0, attempts
                            computer_move_name,final_user_move = '0','0'
                            over=[]
                        else:
                            break

                    # Simultaneously we will get total score of user and computer with every attempt
                    computer_score +=int(computer_move_name)
                    user_score += int(final_user_move)
                    # Changes the color of rectangle as long as hand is inside the rectangle frame.
                    rect_color = (0, 250, 0)
                    de = deque(['nothing'] * 15, maxlen=smooth_factor)

        if final_user_move == '0':           
            hand_inside = False
            rect_color = (255, 0, 0)
 
            # At the end of total attempts, it will show the results of game and ask to play again
            if total_attempts == 0:
                play_again = show_winner(user_score, computer_score)
                # If the user pressed Enter then restart the game by re initializing all variables
                if play_again:
                    user_score, computer_score, total_attempts = 0, 0, attempts
                    over=[]
                # Otherwise quit the program.
                else:
                        break
           
    
     
   
    # This is where all annotation is happening.
    cv2.putText(frame, "Your Move: " + final_user_move,
                    (420, 270), cv2.FONT_HERSHEY_TRIPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, "Computer's Move: " + computer_move_name,
                (2, 270), cv2.FONT_HERSHEY_TRIPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, "Your Score: " + str(user_score),
                    (420, 300), cv2.FONT_HERSHEY_TRIPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, "Computer Score: " + str(computer_score),
                    (2, 300), cv2.FONT_HERSHEY_TRIPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, "Attempts left: {}".format(total_attempts), (190, 400), cv2.FONT_HERSHEY_TRIPLEX, 0.7,
                (255,255, 255), 1, cv2.LINE_AA)
    cv2.rectangle(frame, (width - box_size, 0), (width, box_size), rect_color, 2)
    
    # Display the image   
    cv2.imshow("Cricket Game", frame)
    
    # Exit if 'q' is pressed
    k = cv2.waitKey(10)
    if k == ord('q'):
        break

# Relase the camera and destroy all windows.
cap.release()
cv2.destroyAllWindows()



Ball 1
User move :  1
Computer move :  5

Ball 2
User move :  2
Computer move :  6

Ball 3
User move :  3
Computer move :  1

Ball 4
User move :  4
Computer move :  1

Ball 5
User move :  6
Computer move :  2

Ball 6
User move :  5
Computer move :  6

Total User Score :  21
Total Computer Score :  21

Tie

Ball 1
User move :  3
Computer move :  4

Ball 2
User move :  6
Computer move :  2

Ball 3
User move :  3
Computer move :  5

Ball 4
User move :  1
Computer move :  2

Ball 5
User move :  1
Computer move :  2

Please do not repeat number

Please do not repeat number

Ball 6
User move :  4
Computer move :  2

Total User Score :  18
Total Computer Score :  17

User won

Ball 1
User move :  1
Computer move :  4

Ball 2
User move :  1
Computer move :  4

Ball 3
User move :  2
Computer move :  6

Ball 4
User move :  2
Computer move :  4

Please do not repeat number

Ball 5
User move :  3
Computer move :  1

Please do not repeat number

Ball 6
User move :  4
Computer move :  6

Total User